# 한국어 챗봇 트랜스포머 구현

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import time
import json
import requests
from tqdm import tqdm
import pickle
import datetime
from sklearn.model_selection import train_test_split

이 노트북은 한국어 챗봇 데이터셋(https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv)을 기반으로 트랜스포머 모델을 구현합니다.

## Step 1. 데이터 수집하기

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"
train_data = pd.read_csv(url)

train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
print("챗봇 샘플의 개수 :", len(train_data))

챗봇 샘플의 개수 : 11823


## Step 2. 데이터 전처리하기

In [4]:
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence


def load_conversations(questions, answers):
    inputs, outputs = [], []

    for question, answer in zip(questions, answers):
        inputs.append(preprocess_sentence(question))
        outputs.append(preprocess_sentence(answer))
    return inputs, outputs

In [6]:
# 원본 데이터에서 질문과 답변 추출
raw_questions = train_data["Q"]
raw_answers = train_data["A"]

# 전처리된 질문과 답변 리스트 생성
questions, answers = load_conversations(raw_questions, raw_answers)

In [7]:
print("전처리 후의 123번째 질문 샘플: {}".format(questions[122]))
print("전처리 후의 123번째 답변 샘플: {}".format(answers[122]))

전처리 후의 123번째 질문 샘플: 걸어 가고 있는데 깜깜해서 무서워
전처리 후의 123번째 답변 샘플: 안전 귀가 하세요 .


In [8]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'ppl 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


## Step 3. SubwordTextEncoder 대신 WordPieceTokenizer + 형태소 분석기(Mecab) 사용하기

•	한국어는 띄어쓰기와 어순이 유동적인 언어이기 때문에, 단순한 Subword 방식보다는 형태소 분석기를 사용하는 것이 더욱 효과적일 수 있음 <br>
•	따라서 여기서는 Mecab을 이용해 형태소 단위로 나눈 후, BertWordPieceTokenizer로 subword 단위를 학습함 <br>
•	이를 통해 보다 의미 단위에 가까운 토큰화를 달성하고, 한국어 문장에 더 잘 맞는 토크나이저를 구성함<br>

In [9]:
from konlpy.tag import Mecab

mecab = Mecab()


def tokenize_with_mecab(sentence):
    return mecab.morphs(sentence)

In [10]:
from tokenizers import BertWordPieceTokenizer
save_dir = os.path.expanduser("~/aiffel/chatbot/wordpiece_mecab")
os.makedirs(save_dir, exist_ok=True)

# mecab 기반 토큰 코퍼스 생성
with open("mecab_token_corpus.txt", "w", encoding="utf-8") as f:
    for q, a in zip(questions, answers):
        f.write(" ".join(tokenize_with_mecab(q)) + "\n")
        f.write(" ".join(tokenize_with_mecab(a)) + "\n")

# WordPiece 토크나이저 학습
tokenizer = BertWordPieceTokenizer(lowercase=False)
tokenizer.train(
    files=["mecab_token_corpus.txt"],
    vocab_size=8000,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
)

# 저장 (같은 디렉토리에)
tokenizer.save_model(save_dir)

['/aiffel/aiffel/chatbot/wordpiece_mecab/vocab.txt']

In [11]:
# tokenizer.json도 별도로 저장
tokenizer.save(os.path.join(save_dir, "tokenizer.json"))

In [12]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file(
    os.path.expanduser("~/aiffel/chatbot/wordpiece_mecab/tokenizer.json")
)
output = tokenizer.encode("안녕! 오늘 기분 어때?")
print("Token IDs:", output.ids)
print("Tokens:", output.tokens)

Token IDs: [2205, 5, 1798, 1887, 2020, 21]
Tokens: ['안녕', '!', '오늘', '기분', '어때', '?']


In [13]:
START_TOKEN = "[CLS]"
END_TOKEN = "[SEP]"

# 특수 토큰 ID 가져오기
START_TOKEN_ID = tokenizer.token_to_id(START_TOKEN)
END_TOKEN_ID = tokenizer.token_to_id(END_TOKEN)

# 문장 인코딩 함수
def encode_sentence(sentence):
    tokens = tokenize_with_mecab(sentence)
    token_ids = tokenizer.encode(" ".join(tokens)).ids
    return [START_TOKEN_ID] + token_ids + [END_TOKEN_ID]


# 질문과 답변 각각 정수 인코딩
input_ids = [encode_sentence(q) for q in questions]
target_ids = [encode_sentence(a) for a in answers]

# 타겟 데이터에서 decoder_input / decoder_target 분리
decoder_input_ids = [tokens[:-1] for tokens in target_ids]  # [CLS] ~ 마지막-1
decoder_target_ids = [tokens[1:] for tokens in target_ids]  # 두 번째 ~ [SEP]

# 패딩 (최대 길이 맞춰줌)
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 40

encoder_input = pad_sequences(input_ids, maxlen=MAX_LENGTH, padding="post")
decoder_input = pad_sequences(decoder_input_ids, maxlen=MAX_LENGTH, padding="post")
decoder_target = pad_sequences(decoder_target_ids, maxlen=MAX_LENGTH, padding="post")

# Tensorflow Dataset 생성
import tensorflow as tf

BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices(
    ({"encoder_input": encoder_input, "decoder_input": decoder_input}, decoder_target)
)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [14]:
print(f'질문 토큰화 결과 크기: {len(encoder_input)}')
print(f'대답 토큰화 결과 크기: {len(decoder_input)}')

질문 토큰화 결과 크기: 11823
대답 토큰화 결과 크기: 11823


## Step 4. 모델 구성하기

In [15]:
# 위치 인코딩
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # sin 함수는 짝수 인덱스에 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # cos 함수는 홀수 인덱스에 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [16]:
# 어텐션 마스킹 함수
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [17]:
# 스케일드 닷 프로덕트 어텐션
def scaled_dot_product_attention(q, k, v, mask):
    # Q와 K의 곱셈 연산
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # 스케일링
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # 마스킹 적용
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # 소프트맥스로 어텐션 가중치 계산
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    # 가중치와 V 곱하기
    output = tf.matmul(attention_weights, v)

    return output, attention_weights

In [18]:
# 멀티헤드 어텐션 레이어
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # 스케일드 닷 프로덕트 어텐션 적용
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [19]:
# 포지션 와이즈 피드 포워드 네트워크
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [20]:
# 인코더 레이어
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(v=x, k=x, q=x, mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

In [21]:
# 디코더 레이어
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(v=x, k=x, q=x, mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            v=enc_output, k=enc_output, q=out1, mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

In [22]:
# 인코더
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                              self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        # 입력을 임베딩
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x=x, training=training, mask=mask)

        return x  # (batch_size, input_seq_len, d_model)

In [23]:
# 디코더
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](
                x=x,
                enc_output=enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask
            )

            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

In [24]:
# 패딩 마스크 생성: 입력 시퀀스에서 패딩 토큰(0)에 해당하는 위치를 마스킹
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (배치 크기, 1, 1, 시퀀스 길이)

# 미래 토큰 마스킹 (Look-ahead mask): 디코더가 미래의 토큰을 보지 못하게 막음
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (시퀀스 길이, 시퀀스 길이)

# 전체 마스크 생성 함수: 인코더/디코더 패딩 마스크와 Look-ahead 마스크 결합
def create_masks(inp, tar):
    # 인코더 입력에 대한 패딩 마스크
    enc_padding_mask = create_padding_mask(inp)

    # 디코더 입력에 대한 look-ahead 마스크와 패딩 마스크 결합
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    # 디코더의 인코더 출력에 적용할 패딩 마스크
    dec_padding_mask = create_padding_mask(inp)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [25]:
# 트랜스포머 모델
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                             target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):  # ✅ 클래스 안으로 들여쓰기!
        # 리스트/튜플 구조일 때: enc_input, dec_input 나눔
        enc_input, dec_input = inputs

        # 마스크 생성
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(enc_input, dec_input)

        # 인코더
        enc_output = self.encoder(
            x=enc_input,
            training=training,
            mask=enc_padding_mask
        )

        # 디코더
        dec_output, attention_weights = self.decoder(
            x=dec_input,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )

        # 출력층
        final_output = self.final_layer(dec_output)
        return final_output

In [26]:
VOCAB_SIZE = tokenizer.get_vocab_size()

In [27]:
# 학습 파라미터 설정
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

# 모델 인스턴스 생성
model = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=VOCAB_SIZE,
    target_vocab_size=VOCAB_SIZE,
    pe_input=MAX_LENGTH,
    pe_target=MAX_LENGTH,
    rate=dropout_rate
)

In [28]:
# 손실 함수 정의
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.keras.losses.sparse_categorical_crossentropy(
        real, pred, from_logits=True)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

# 정확도 계산 함수 정의 
def accuracy(real, pred):
    accuracies = tf.equal(real, tf.cast(tf.argmax(pred, axis=2), dtype=tf.int32))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)

    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

In [29]:
# 학습률 스케줄러 정의
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        # step을 float32로 명시적 변환 추가
        step = tf.cast(step, tf.float32)

        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# 학습률 설정 및 옵티마이저 정의
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 체크포인트 설정
checkpoint_path = './checkpoints/transformer'
ckpt = tf.train.Checkpoint(transformer=model, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [30]:
# 학습 함수 정의
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    with tf.GradientTape() as tape:
        predictions = transformer(
            {
                'inputs': inp,
                'dec_inputs': tar_inp
            },
            training=True
        )

        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(tar_real, predictions))

In [31]:
import time
from tqdm.notebook import tqdm
import tensorflow as tf

# ⬇️ 메트릭 초기화
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# ✅ train_step 함수 정의
@tf.function
def train_step(enc_inp, dec_inp, target):
    with tf.GradientTape() as tape:
        # 모델 forward pass
        predictions = model([enc_inp, dec_inp], training=True)
        # 손실 계산
        loss = loss_function(target, predictions)

    # 역전파
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 메트릭 업데이트
    train_loss.update_state(loss)
    train_accuracy.update_state(accuracy(target, predictions))

# ⬇️ 학습 시작
EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_state()
    train_accuracy.reset_state()

    print(f"에포크 {epoch + 1}/{EPOCHS} 시작...")

    for (batch, (inp, tar)) in tqdm(enumerate(dataset), total=len(dataset)):
        # ⛳️ tar는 decoder_target이므로 그냥 넘기면 됩니다
        train_step(inp['encoder_input'], inp['decoder_input'], tar)

    epoch_time = time.time() - start
    print(f'에포크 {epoch + 1} 완료 - 손실: {train_loss.result():.4f}, 정확도: {train_accuracy.result():.4f}, 시간: {epoch_time:.2f}초')

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'에포크 {epoch + 1}에 체크포인트 저장: {ckpt_save_path}')

    print("-" * 50)

에포크 1/50 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 1 완료 - 손실: 8.2714, 정확도: 0.0794, 시간: 21.29초
--------------------------------------------------
에포크 2/50 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 2 완료 - 손실: 6.5577, 정확도: 0.1700, 시간: 7.48초
--------------------------------------------------
에포크 3/50 시작...


...

에포크 48 완료 - 손실: 0.3232, 정확도: 0.9054, 시간: 7.59초
--------------------------------------------------
에포크 49/50 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 49 완료 - 손실: 0.3185, 정확도: 0.9075, 시간: 7.59초
--------------------------------------------------
에포크 50/50 시작...


  0%|          | 0/185 [00:00<?, ?it/s]

에포크 50 완료 - 손실: 0.3099, 정확도: 0.9092, 시간: 7.58초
에포크 50에 체크포인트 저장: ./checkpoints/transformer/ckpt-10
--------------------------------------------------


## Step 5. 모델 평가하기

In [32]:
def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    tokenized = tokenizer.encode(" ".join(tokenize_with_mecab(sentence))).ids
    encoder_input = [START_TOKEN_ID] + tokenized + [END_TOKEN_ID]
    encoder_input = tf.convert_to_tensor([encoder_input], dtype=tf.int32)
    
    decoder_input = tf.convert_to_tensor([[START_TOKEN_ID]], dtype=tf.int32)
    
    for _ in range(MAX_LENGTH):
        # 모델 입력을 튜플로 전달
        predictions = model([encoder_input, decoder_input], training=False)
        predicted_id = tf.cast(tf.argmax(predictions[:, -1:, :], axis=-1), dtype=tf.int32)  # int32로 명시적 캐스팅
        
        if predicted_id.numpy()[0][0] == END_TOKEN_ID:
            break
            
        decoder_input = tf.concat([decoder_input, predicted_id], axis=-1)
    
    output_ids = decoder_input.numpy()[0].tolist()
    output_ids = [i for i in output_ids if i != START_TOKEN_ID and i != END_TOKEN_ID]
    
    return tokenizer.decode(output_ids)

In [33]:
START_TOKEN_ID = tokenizer.token_to_id("[CLS]")
END_TOKEN_ID = tokenizer.token_to_id("[SEP]")

In [34]:
# 토큰 ID 확인
print("START_TOKEN_ID:", START_TOKEN_ID, "타입:", type(START_TOKEN_ID))
print("END_TOKEN_ID:", END_TOKEN_ID, "타입:", type(END_TOKEN_ID))

START_TOKEN_ID: 2 타입: <class 'int'>
END_TOKEN_ID: 3 타입: <class 'int'>


In [35]:
def chat_response(sentence):
    result = evaluate(sentence)
    return result.strip() 

In [36]:
def chat_response(sentence):
    result = evaluate(sentence)
    # 과도한 공백 제거 (연속된 공백을 하나로 변경)
    result = re.sub(r'\s+', ' ', result.strip())
    return result

In [37]:
# 몇 가지 질문에 대한 응답 테스트
test_questions = [
    '내일 내 운세 알려줄래?',
    '권정원은 옆사람과 친해질 수 있을까요?',
    '띄어쓰기를 왜 그렇게 하세요?',
    '난 이제 지쳤어요 그래서 더이상 못하겠어요 알겠어요?'
    
]

for question in test_questions:
    print('질문:', question)
    print('응답:', chat_response(question))
    print()


질문: 내일 내 운세 알려줄래?
응답: 다른 곳 에 쓰 려고 운 을 아껴 뒀 나 봐요.

질문: 권정원은 옆사람과 친해질 수 있을까요?
응답: 건들 지 않 는 선 에서 물 어 보 세요.

질문: 띄어쓰기를 왜 그렇게 하세요?
응답: 쓰 면서 정리 가 되 기 도 하 죠.

질문: 난 이제 지쳤어요 그래서 더이상 못하겠어요 알겠어요?
응답: 가치관 이 중요 한 거 같 아요.

질문: 이따 뭐먹지?
응답: 잘 해결 될 거 예요.



### 띄어쓰기 패턴이 특이한 것은 형태소 분석기(mecab)와 토크나이저의 특성 때문입니다.<br>
### 한국어 문장을 형태소 단위로 분리하면서, 조사나 어미가 독립적인 토큰으로 분리되기 때문에 이런 현상이 발생합니다. 예를 들어 "이유가"는 "이유" + "가"로 분리되어 토큰화되고, 디코딩 시 다시 결합할 때 공백이 생깁니다.

회고: 토크나이저 다른 거 썼더니 이렇게 되었다. 아쉽다. transformer 이론도 어렵지만 실습도 어렵다🥲🥲

## 7. 모델 개선 및 확장 방안

현재 구현한 챗봇 모델을 더 개선하기 위한 방안들:

1. **데이터 증강**: 기존 데이터셋을 활용하여 백트랜슬레이션, 동의어 치환 등을 통해 데이터를 증강
2. **사전 학습 모델 활용**: KoBERT, KoGPT 등 한국어 사전 학습 모델을 활용하여 전이학습 적용
3. **더 큰 모델과 더 긴 학습**: 모델 크기를 키우고 더 오래 학습하여 성능 향상
4. **문맥 유지**: 이전 대화 기억하여 문맥을 유지하는 기능 추가
5. **다양한 응답 생성**: 빔 서치, Top-k/Top-p 샘플링 등을 통해 더 다양한 응답 생성